In [2]:
import pandas as pd
import pickle as pkl
import numpy as np
import re
import json
from openai import OpenAI

pd.set_option('display.max_colwidth', None)  # 열의 너비 제한 없애기

In [ ]:
def evmine_result(json_file_path, mapping_file_path, min_num=10):
    # output 파일 읽기
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    print(f'Total number of events:{len(data)}')

    # mapping 파일 읽기
    with open(mapping_file_path, 'r', encoding='utf-8') as infile:
        mapping = infile.readlines()
    parsed_data = []
    for line in mapping:
        org_id, id = line.strip().split(': ')
        parsed_data.append({'org_id': org_id, 'id': int(id)})
    mapping = pd.DataFrame(parsed_data)
    
    # news 파일 읽기
    with open("/home/dieselim/main/data/preprocessed_news_revised.pkl", 'rb') as file:
        news = pkl.load(file)
    news = pd.DataFrame(news)

    # 각각의 내부 리스트에 대해 필터링
    filtered_dfs = []
    for sublist in data:
        if len(sublist)>=min_num:
            sublist_str = list(sublist)
            org_ids = mapping[mapping['id'].isin(sublist_str)]
            filtered_news = news[news['news_id'].isin(org_ids['org_id'])]
            filtered_dfs.append(filtered_news)
    print(f'Number of events above {min_num}: {len(filtered_dfs)}')
    return filtered_dfs

def get_topic_from_titles(titles):
    client = OpenAI(api_key='')
    prompt = f"The given news titles have been clustered based on a common topic. The titles are in Korean, and your answer should also be in Korean. All the news have been collected using the keyword 'semiconductor'. Identify the event that occurred based on these titles, trying to encompass nearly all news titles. Finally, provide only the name of the event in a detailed yet concise one phrase, and nothing else. Do not include any explanations or descriptions. \n\n{titles}\n\n"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        ],
    )
    topic = response.choices[0].message.content
    return topic

def evmine_named_result(filtered_dfs):
    final_topic = {}
    for df in filtered_dfs:
        titles = "\n".join(df['title'].head(100).tolist())
        topic = get_topic_from_titles(titles)
        sub_dic = {}
        sub_dic['news'] = df[['yearmonth', 'title', 'sentence']]
        sub_dic['period_min'] = df['yearmonth'].min()
        sub_dic['period_max'] = df['yearmonth'].max()
        sub_dic['news_count'] = len(df)
        final_topic[topic] = sub_dic
        
    # 딕셔너리를 'period_min' 값으로 정렬
    sorted_final_topic = dict(sorted(final_topic.items(), key=lambda item: item[1]['period_min']))
    
    return sorted_final_topic

def evmine_save(final_topic, output_path):
    # 딕셔너리를 pickle 형식으로 저장
    with open(output_path, 'wb') as file:
        pkl.dump(final_topic, file)
    print(f'Named events saved as: {output_path}')
        
def EvMine(json_file_path, mapping_file_path, min_num, output_path):
    filtered_dfs = evmine_result(json_file_path, mapping_file_path, min_num)
    final_topic = evmine_named_result(filtered_dfs)
    evmine_save(final_topic, output_path)
    return final_topic

In [3]:
def print_results(final_topic, news_num):   
    for idx, topic in enumerate(final_topic.keys()):
        subdic = final_topic[topic]
        print(f"Name of Topic {idx}: {topic}")
        print(f"Time Period: {subdic['period_min']} ~ {subdic['period_max']}")
        print(f"News Count: {len(subdic['news'])}")
        print(subdic['news'][['yearmonth', 'title']].head(news_num))
        print("\n")

def print_by_news_count(final_topic, news_num): 
    sorted_topics = sorted(final_topic.items(), key=lambda item: len(item[1]['news']), reverse=True)
      
    for idx, (topic, subdic) in enumerate(sorted_topics):
        subdic = final_topic[topic]
        print(f"Name of Topic {idx}: {topic}")
        print(f"Time Period: {subdic['period_min']} ~ {subdic['period_max']}")
        print(f"News Count: {len(subdic['news'])}")
        print(subdic['news'][['yearmonth', 'title']].head(news_num))
        print("\n")

def print_by_news_count_only_title(final_topic): 
    sorted_topics = sorted(final_topic.items(), key=lambda item: len(item[1]['news']), reverse=True)
      
    for idx, (topic, subdic) in enumerate(sorted_topics):
        subdic = final_topic[topic]
        print(f"Name of Topic {idx}: {topic}")
        print(f"Time Period: {subdic['period_min']} ~ {subdic['period_max']}  /  News Count: {len(subdic['news'])}\n")

In [4]:
path = '/home/dieselim/main/data/evmine_data/re_frac6/'
json_file_path = path+'output.json'
mapping_file_path = path+'mapping.final.txt'
min_num = 50
output_path = path + 'EvMine.pkl'
final_topic_merged = EvMine(json_file_path, mapping_file_path, min_num, output_path)
print_results(final_topic_merged,5)

Total number of events:284
Number of events above 50: 106
Named events saved as: /home/dieselim/main/data/evmine_data/re_frac6/EvMine.pkl
Name of Topic 0: 삼성전자 반도체 및 모바일 사업의 실적 변동.
Time Period: 2010-01-01 ~ 2023-06-30
News Count: 6900
       yearmonth                                        title
6164  2022-01-31    "1000원 팔아 500원 남겼다"…마진율 상상초월 반도체 업계는 딴 세상
6252  2019-04-30  "10분기 만에 최저"삼성전자, 1분기 매출 52조원영업익 6.2조원 (상보)
6259  2020-09-21        "10조? 11조?"삼성전자, 7분기 만에 영업이익 10조원 넘기나
6338  2022-04-06                 "1Q 잠정실적 발표 시즌 시작…에너지 운송 주목"
6386  2018-12-31                     "1분기 실적 호전 대형 IT기업 주목해야"


Name of Topic 1: 이재용 삼성전자 회장 취임 및 삼성의 미래를 위한 경영 전략 구상.
Time Period: 2010-01-04 ~ 2023-06-29
News Count: 3127
       yearmonth                                title
1541  2021-11-19     광폭 행보 나선 이재용 부회장...제2 반도체 신화 일군다
4036  2022-10-25   이건희 2주기 참석한 삼성이재용, '뉴 삼성' 회장 승진 임박
4050  2020-10-28              이건희 회장 발인서초사옥 앞 무거운 분위기
5734  2021-12-20            " 시장 키우자"삼성전자, 중국사업혁신팀 신설
6243  2021-1

In [64]:
path = '/home/dieselim/main/data/evmine_data/re_frac1/'
json_file_path = path+'output.json'
mapping_file_path = path+'mapping.final.txt'
min_num = 15
output_path = path + 'EvMine.pkl'
final_topic_merged = EvMine(json_file_path, mapping_file_path, min_num, output_path)
print_results(final_topic_merged,5)

Total number of events:427
Number of events above 15: 102
Named events saved as: /home/dieselim/main/data/evmine_data/re_frac1/EvMine.pkl
Name of Topic 0: 르네사스 공장 6월 재가동 및 삼성전자 1분기 실적 발표
Time Period: 2010-08-02 ~ 2011-05-08
News Count: 25
         yearmonth                                 title
2142    2011-04-13           르네사스, 6월 공장 재가동..예정보다 한달 빨라
22539   2011-04-25         "주성엔지니어링, 하반기 연간 최대 실적 예상"-대우
41232   2010-10-07          (스톡톡스)"삼성電, 3Q 예상수준..4Q가 걱정"
105503  2011-01-30  [1분기 실적 4대 포인트] ① 중소형IT株 영업이익 `퀀텀점프`
160898  2011-04-29     [스톡톡스]"삼성電, 1Q 예상수준..2Q 단말기 등 부진"


Name of Topic 1: 삼성-애플 특허전쟁
Time Period: 2011-02-09 ~ 2011-10-24
News Count: 63
        yearmonth                            title
7808   2011-08-08          "IT업계, 사상 최대 특허전쟁 진행 중"
15497  2011-09-17         "삼성, 아이폰용 모바일 CPU 독점 깨져"
19038  2011-06-28  "애플 차세대 아이패드 CPU 'A6' 삼성부품 줄일듯"
19132  2011-04-20       "애플, 안드로이드 진영과 '특허 전쟁' 벌여"
30246  2011-04-19          '동지에서 적으로'…애플·삼성 소송戰 돌입


Name of Topic 2: 삼성 반도체 백혈병 근

# 최종 리스트 만들기 (걸러내기 전)

### 병합 필요

In [11]:
def load_df(num):
    output_path = '/home/dieselim/main/data/evmine_data/bydate/'+str(num)+'/EvMine.pkl'
    # 저장된 딕셔너리를 불러오기
    with open(output_path, 'rb') as file:
        loaded_df = pkl.load(file)
    return loaded_df

In [7]:
loaded_1 = load_df('1_re')
loaded_2 = load_df(2)
loaded_3 = load_df(3)
loaded_4 = load_df(4)
loaded_5 = load_df(5)
loaded_6 = load_df(6)
loaded_7 = load_df(7)

loaded_final_topic = {**loaded_1, **loaded_2, **loaded_3, **loaded_4, **loaded_5, **loaded_6, **loaded_7}

In [8]:
df1 = pd.DataFrame({
    'Topic Num': range(1, len(loaded_final_topic)+1),
    'Event Title': loaded_final_topic.keys(), 
    'Start': [subdic[1]['period_min'] for subdic in loaded_final_topic.items()],
    'End': [subdic[1]['period_max'] for subdic in loaded_final_topic.items()],
    'News Count': [len(subdic[1]['news']) for subdic in loaded_final_topic.items()]
})

### 병합 필요 x

In [1]:
def load_df(path):
    output_path = path+'/EvMine.pkl'
    # 저장된 딕셔너리를 불러오기
    with open(output_path, 'rb') as file:
        loaded_df = pkl.load(file)
    return loaded_df

In [22]:
loaded_1 = load_df('/home/dieselim/main/data/evmine_data/re_frac1')
loaded_2 = load_df('/home/dieselim/main/data/evmine_data/US-China-1823')

In [31]:
print_results(loaded_1,5)

Name of Topic 0: 삼성전자 실적 부진
Time Period: 2010-08-02 ~ 2011-05-08
News Count: 25
         yearmonth                                 title
2142    2011-04-13           르네사스, 6월 공장 재가동..예정보다 한달 빨라
22539   2011-04-25         "주성엔지니어링, 하반기 연간 최대 실적 예상"-대우
41232   2010-10-07          (스톡톡스)"삼성電, 3Q 예상수준..4Q가 걱정"
105503  2011-01-30  [1분기 실적 4대 포인트] ① 중소형IT株 영업이익 `퀀텀점프`
160898  2011-04-29     [스톡톡스]"삼성電, 1Q 예상수준..2Q 단말기 등 부진"


Name of Topic 1: 애플-삼성 특허전쟁
Time Period: 2011-02-09 ~ 2011-10-24
News Count: 63
        yearmonth                            title
7808   2011-08-08          "IT업계, 사상 최대 특허전쟁 진행 중"
15497  2011-09-17         "삼성, 아이폰용 모바일 CPU 독점 깨져"
19038  2011-06-28  "애플 차세대 아이패드 CPU 'A6' 삼성부품 줄일듯"
19132  2011-04-20       "애플, 안드로이드 진영과 '특허 전쟁' 벌여"
30246  2011-04-19          '동지에서 적으로'…애플·삼성 소송戰 돌입


Name of Topic 2: 삼성전자의 백혈병 근로자 산재 인정 사건
Time Period: 2011-06-23 ~ 2011-09-05
News Count: 25
         yearmonth                            title
16284   2011-06-23             "삼성전자 백혈병 근로자

In [32]:
print_results(loaded_2,5)

Name of Topic 0: SK하이닉스 및 도시바의 엘피다 인수전 포기 및 마이크론의 엘피다 인수
Time Period: 2012-04-05 ~ 2012-05-06
News Count: 34
        yearmonth                          title
8385   2012-04-22    "SK하이닉스, 엘피다 인수에 최대 2조원 제시"
11912  2012-04-24    "도시바, 엘피다 2차 입찰 참여 않기로"(종합)
11913  2012-04-24     "도시바, 엘피다 입찰 불참할 듯" - 마이니치
11914  2012-04-24            "도시바, 엘피다 입찰 참여 포기"
11917  2012-04-05  "도시바, 하이닉스와 엘피다 공동 인수 검토"(상보)


Name of Topic 1: 박근혜 대통령의 중국 국빈 방문 및 경제사절단 파견
Time Period: 2013-06-09 ~ 2013-07-01
News Count: 31
         yearmonth                            title
86743   2013-06-26            SK 김창근 의장 방중…글로벌 경영행보
86744   2013-06-26   SK 김창근 의장, 朴 대통령 경제사절단으로 중국 방문
86750   2013-06-26   SK 김창근 의장, 글로벌 경영 활동…中 산업현장 시찰
86751   2013-06-26      SK 김창근 의장, 중국에서 글로벌경영 활동 전개
135210  2013-06-30  [뉴스 분석] 新 한·중 20년 ‘신뢰의 여정’ 시작됐다


Name of Topic 2: 수출 감소 및 경제 위기
Time Period: 2016-04-01 ~ 2017-02-28
News Count: 42
        yearmonth                                 title
27750  2016-04-22      'PC 시장 침체' 인텔·퀄컴 IT업계

In [68]:
df1 = pd.DataFrame({
    'Topic Num': range(1, len(loaded_1)+1),
    'Event Title': loaded_1.keys(), 
    'Start': [subdic[1]['period_min'] for subdic in loaded_1.items()],
    'End': [subdic[1]['period_max'] for subdic in loaded_1.items()],
    'News Count': [len(subdic[1]['news']) for subdic in loaded_1.items()]
})

df2 = pd.DataFrame({
    'Topic Num': range(len(loaded_1)+1, len(loaded_1)+len(loaded_2)+1),
    'Event Title': loaded_2.keys(), 
    'Start': [subdic[1]['period_min'] for subdic in loaded_2.items()],
    'End': [subdic[1]['period_max'] for subdic in loaded_2.items()],
    'News Count': [len(subdic[1]['news']) for subdic in loaded_2.items()]
})

In [45]:
df1

,Topic Num,Event Title,Start,End,News Count
0,1,삼성전자 실적 부진,2010-08-02,2011-05-08,25
1,2,애플-삼성 특허전쟁,2011-02-09,2011-10-24,63
2,3,삼성전자의 백혈병 근로자 산재 인정 사건,2011-06-23,2011-09-05,25
3,4,삼성전자 반도체 및 모바일 부문 실적 호조,2011-09-21,2013-10-07,67
4,5,스티브 잡스 추도식 참석,2011-10-03,2011-11-06,16
...,...,...,...,...,...
97,98,이재용-손정의 회동 및 삼성의 ARM 인수 논의,2022-09-01,2022-10-06,40
98,99,2030 부산 엑스포 유치 활동,2022-11-11,2023-06-22,55
99,100,한일 정상회담 및 경제협력 행사,2023-03-08,2023-05-26,41
100,101,윤석열 대통령의 미국 국빈 방문 및 경제 사절단 활동,2023-03-08,2023-06-23,54


In [69]:
final_df = pd.concat([df1,df2], axis=0)
final_df = final_df.sort_values(by = 'Start')
final_df

,Topic Num,Event Title,Start,End,News Count
0,1,르네사스 공장 6월 재가동 및 삼성전자 1분기 실적 발표,2010-08-02,2011-05-08,25
0,103,주성엔지니어링 3분기 실적 발표 및 태양광 산업 성장,2010-10-05,2010-11-05,16
1,104,삼성전자 2010년 임원 승진 인사,2010-12-08,2010-12-13,19
1,2,삼성-애플 특허전쟁,2011-02-09,2011-10-24,63
2,105,삼성 반도체 CIO 포럼 독일 개최,2011-04-14,2011-04-15,24
...,...,...,...,...,...
93,196,미국 반도체 지원법(칩스법),2023-02-08,2023-06-30,76
94,197,윤석열 대통령 국빈 방미,2023-03-07,2023-06-03,90
100,101,윤석열 대통령 국빈 방미 및 경제사절단 활동,2023-03-08,2023-06-23,54
99,100,한일 정상회담 및 경제인 협력 행사,2023-03-08,2023-05-26,41


In [70]:
with pd.ExcelWriter('/home/dieselim/main/data/EvMineFinlaResult/re_frac1+US-China.xlsx') as writer:
    final_df.to_excel(writer, sheet_name='이벤트 리스트', index=False)

In [23]:
df3 = pd.DataFrame()
for idx, subdic in enumerate(loaded_1.items()):
    subdic[1]['news']['topic number'] = idx+1
    sub_df = subdic[1]['news'].reset_index(drop=True)
    sub_df.reset_index(inplace = True)
    sub_df['index'] += 1
    
    df3 = pd.concat([df3, sub_df], axis=0)

df4 = pd.DataFrame()
for idx, subdic in enumerate(loaded_2.items()):
    subdic[1]['news']['topic number'] = len(loaded_1)+1+idx
    sub_df = subdic[1]['news'].reset_index(drop=True)
    sub_df.reset_index(inplace = True)
    sub_df['index'] += 1
    
    df4 = pd.concat([df4, sub_df], axis=0)

In [7]:
df4.head(2)

,index,yearmonth,title,sentence,topic number
0,1,2010-10-25,"""주성엔지니어링, 태양광이 성장 주도""[미래에셋]","[(서울=연합인포맥스) 장용욱 기자 = 주성엔지니어링의 실적이 태양광 부문의 지속적인 성장세에 힘입어 꾸준히 증가할 것으로 전망됐다., 박주비 미래에셋증권 연구원은 25일 기업분석 보고서를 통해 ""주성엔지니어링은박막과 결정형 태양전지 장비 모두를 제작할 수 있다""며 ""따라서 중국과 미국의 대규모 발전소 증가에 따른 박막시장 고성장으로 혜택을 받을 것""이라고 분석했다., 특히 최대 경쟁자였던 어플라이드 머티어리얼스(Applied Materials)가 비정질 실리콘 박막형 시장에서 철수한 것이 주성엔지니어링에는 더 큰 기회가 될 것으로 보인다., 중국 주요 전력회사로부터 수주한 경험이 있는 주성엔지니어링이 시장을 독식할수 있을 것으로 분석되기 때문이다., 박 연구원은 ""주성엔지니어링은 대면적 LCD용 플라즈마화학증착장비(PECVD)로 새로운 시장도 공략하고 있다""며 ""이 덕분에 태양전지 부문 경쟁력은 지속적으로 상승할 것""이라고 전망했다., 이를 바탕으로 주성엔지니어링 태양광 부문의 내년 매출액은 전년보다 50%나 상승한 3천억원 수준일 것으로 예상됐다., 전체 매출에서 55%가량을 차지하게 되는 것이다., 박 연구원은 ""이 외에도 반도체 ALD 및 LCD PECVD 장비의 안정적인 실적 유지도예상된다""며 ""내년 매출액과 영업익은 각각 5천439억원, 768억원에 이를 것""이라고내다봤다., yujang@yna.co.kr(끝)<저작권자(C)연합인포맥스뉴스. 무단 전재-재배포 금지., >]",103
1,2,2010-10-13,10대 핵심기술 개발수출 산업화등 다각 지원,"[5년간 신재생산업에 민관합동 40조 투입태양전지바이오연료 등 차세대 기술 집중투자수출 1억 이상 달성할 스타기업도 50개 육성반도체, 조선, 자동차 등은 수출의존도가 90%에 달하는 우리나라를 먹여 살린 주력 수출 산업이다., 하지만 21세기 저탄소 녹색성장 시대를 맞아 신재생에너지 세계시장이 폭발적으로 성장하면서 태양광, 풍력 등의 산업군이 새롭게 떠오르는 추세다., 미국, 유럽연합(EU) 등의 선진국뿐 아니라 중국이 관련 분야에서 급부상하고 있는 현실에서 우리의 태양광ㆍ풍력 등 신재생에너지 산업은 상대적으로 뒤쳐졌다는 평가다., 이에 따라 해외시장 선점 및 글로벌 경쟁력 확보를 위해 보다 구체적인 로드맵을 마련해야 할 필요성이 높아졌고, 정부는 민간과 함께 40조원을 투입해 신재생에너지 분야를 육성하기로 했다., 지식경제부의 한 관계자는 ""지난해 말 현재 146개 신재생에너지 제조사 중 116개가 중소ㆍ중견기업으로 기술경쟁력 미흡, 내수시장 창출 한계, 글로벌 기업 부재, 금융ㆍ세제ㆍ인력 등의 기업성장 지원인프라 취약이라는 문제를 안고 있어 이번 대책을 마련했다""고 말했다., 떠오르는 신재생에너지시장, 우리 경쟁력은 미흡해=차세대 에너지원으로서 신재생에너지 시장의 성장 추세는 가히 폭발적이다., 지난 5년간 연평균 28.2% 성장해왔고 오는 2015년에는 그 규모가 4,000억달러, 2020년에는 1조달러까지 급증할 것으로 예상된다., 이미 선도그룹을 형성하고 있는 미국, EU, 일본 외에도 중국은 태양광과 풍력 분야에 과감한 투자로 시장을 주도하고 있다., 우리와 실질적 경쟁자인 중국은 2020년까지 신재생에너지 비중을 15%로 올린다는 계획아래, 지난해에만 346억달러를 태양광과 풍력 등의 산업에 집중 투자했다., 중국은 오는 2020년까지 7,400억달러를 신재생에너지산업에 투자할 계획이다., 반면 우리는 최근 3년간 신재생에너지 분야에 2조57억원을 투자했지만 시장점유율은 4~14%대에 그친다., 특히 수출 경쟁력 측면에서 봤을 때 중국에도 한참 뒤쳐져 있는 것이 현실이다., 태양광의 경우 결정질 태양전지 기준 선진국의 효율은 18% 이상으로 우리(16~18%수준)보다 품질이 앞서있고, 중국은 가격이 와트당 1.35달러 이하로 우리나라(1.35~1.4달러)보다 저렴하다., 풍력도 상업성을 인정받고 있는 해상 풍력의 국산화 수준은 70%에 불과해 산업 육성이 시급하다., 핵심기술 개발에서 수출 산업화까지 지원=정부는 2012년 8대, 2015년 5대 신재생에너지 강국으로 도약한다는 목표로 태양광 분야를 제2의 반도체로, 풍력은 제2의 선박으로 키울 방침이다., 즉, 수출, 고용 등 우리 경제를 선도하는 차세대 주력산업을 신재생에너지산업에서 찾는다는 것이다., 정부는 대형 R&D(연구개발) 위주로 소위 될만한 핵심 기술들을 집중 육성하기로 했다., 먼저 세계시장 선도 10대 핵심기술에 2015년까지 1조5,000억원을 집중 투자한다., 10대 핵심기술은 차세대 태양전지 분야에서 박막, 염료감응, 나노유기 등과 실리콘계 태양전지 고효율화 등 총 4개가 선정됐고, 풍력에서는 해상용 대형풍력 및 부유식 풍력발전 기술 등 2개가 포함됐다., 이 밖에 차세대 수소연료전지, 목질계와 해조류 바이오연료 생산, 수출용 석탄가스화 복합발전시스템도 선정됐다., 중소ㆍ중견기업을 중심으로 한 8대 부품ㆍ소재ㆍ장비 기술개발에도 2015년까지 1조원을 지원해 산업 뿌리를 단단하게 다지고, 내수시장 창출을 위해서는 공공분야를 중심으로 신재생에너지 설비를 설치하는 '10대 그린 프로젝트'를 추진한다., 수출 산업화는 풍력 산업부터 집중한다., 2012년까지 대형 해상풍력발전기 개발을 완료하고, 2013년에는 서남해안권에 100급 실증단지를 구축해 해외진출을 위한 기반을 마련할 방침이다., 또한 2015년까지 수출 1억달러 이상 글로벌 스타 기업 50개를 육성하기로 하고, 중견기업을 중심으로 후보를 선정해 기술개발부터 금융, 마케팅, 인증까지 필요한 지원을 '패키지' 형태로 제공할 계획이다., 핵전쟁에도 안전한 '초호화 지하벙커' 제2의 반도체조선산업에 40조 투자 쏘나타에쿠스 인기 이정도였다, 니 '헉!' 북한 화학탄 125만발이나 소니, 웹서핑되는 구글TV '저가공세'황정원기자 garden@sed.co.kr[알파클럽] 시장을 주도할 업종 및 유망종목 투자설명회 참가 접수![ 인터넷한국일보(www.hankooki.com), 무단 전재 및 재배포 금지]]",103


In [8]:
final_df2 = pd.concat([df3,df4], axis=0)
final_df2.head(2)

,index,yearmonth,title,sentence,topic number
0,1,2011-04-13,"르네사스, 6월 공장 재가동..예정보다 한달 빨라","[- 자동차용 반도체마이크로컨트롤러 생산 나카공장- 기존 예정보다 한달 앞당겨 재가동 가능[이데일리 안혜신 기자] 일본 반도체 업체인 르네사스전자가 지난 11일 발생한 대지진으로 피해를 입은 이바라키현 나카 공장을 오는 6월부터 재가동한다고 13일 니혼게이자이신문이 보도했다., 이는 기존 계획보다 1개월가량 빠른 것이다., 나카 공장은 자동차에 사용되는 반도체와 마이크로컨트롤러(MCU) 등을 대량생산하는 공장으로 지난달 발생한 대지진의 직격타를 맞았다., 르네사스는 전세계 자동차용 마이트로컨트롤러 점유율의 30%를 차지하고 있다., 따라서 이번 대지진 여파로 인한 공장가동 소식이 전해지자 전 세계 자동차 생산에 차질이 이어질 것이라는 우려가 증폭되기도 했다., 나카공장은 르네사스 자동차용 마이크로컨트롤러 생산의 25%가량을 담당하고 있다., 한편 반도체 웨이퍼 제조업체인 썸코(SUMCO) 역시 지진 이후 생산이 중단됐던 요네자와공장 가동 재개를 고려 중이며, 신에츠화학도 1~2주 내 시라카와 공장 가동을 재개할 예정이다., 찌라시는 가라!.., 스마트브리프 하루 두번 무료로 만나보세요., 이데일리ON, 무료로 이용하는 실시간 현재가 조회 1666-2200 이데일리 모바일ㆍ실시간 해외지수/SMS <3993+show/nate/ez-i> 가장 빠른 글로벌 경제뉴스ㆍ금융정보 터미널, 이데일리 MARKETPOINT종합 경제정보 미디어 이데일리 - 무단전재&재배포 금지안방에서 만나는 가장 빠른 경제뉴스ㆍ돈이 되는 재테크정보 - 이데일리TV안혜신 (ahnhye@)]",1
1,2,2011-04-25,"""주성엔지니어링, 하반기 연간 최대 실적 예상""-대우","[<아이뉴스24>[이부연기자] 대우증권은 25일 주성엔지니어링에 대해 하반기 실적 개선이 기대된다며 투자의견 '매수', 목표주가 2만6천원을 제시했다., 송종호 연구원은 ""주성엔지니어링의 1분기 실적은 매출 972억원, 영업이익 75억원으로 예상치에 못미쳤다""면서 ""다만 1분기에 1천471억원을 신규 수주해 향후 성장 잠재력 측면에서는 긍정적""이라고 말했다., 분기 실적은 1분기를 바닥으로 성장세가 지속될 것이라며 하반기에 연간으로는 사상 최대 실적을 경신할 것이라고 내다봤다., 송 연구원은 ""증착 장비에 대한 경쟁력을 바탕으로 반도체와 LCD, 태양전지 장비에서 성장을 이루고 있어 주성엔지니어링의 성장성은 현재 진행형""이라며 ""올해 연간 매출은 전년 대비 28.4% 증가한 5천438억원, 영업이익은 29.5% 늘어난 626억원을 기록할 것""이라고 언급했다., /이부연기자 boo@inews24.comIT는 아이뉴스24 연예ㆍ스포츠는 조이뉴스24새로운 시각 즐거운 게임, 아이뉴스24 게임메일로 보는 뉴스 클리핑, 아이뉴스24 뉴스레터(Copyright 아이뉴스24. 무단전재 및 재배포 금지)]",1


In [11]:
with pd.ExcelWriter('/home/dieselim/main/data/EvMineFinlaResult/output.xlsx') as writer:
    final_df.to_excel(writer, sheet_name='이벤트 리스트', index=False)
    df2.to_excel(writer, sheet_name='뉴스 리스트', index=False)

# 걸러낸 후 진짜 최종 리스트

In [5]:
# Excel 파일의 시트를 읽습니다.
filtered_df = pd.read_excel('/home/dieselim/main/data/EvMineFinlaResult/240802_이벤트 리스트 수정.xlsx', sheet_name='이벤트 리스트')

In [24]:
filtered_df3 = df3[df3['topic number'].isin(filtered_df['Topic Num'])]
filtered_df4 = df4[df4['topic number'].isin(filtered_df['Topic Num'])]

In [25]:
print(len(filtered_df3), len(filtered_df4))

2338 2733


In [20]:
filtered_df3['topic number'].unique()

array([ 2,  7, 19, 26, 35, 36, 38, 39, 40, 47, 52, 55, 56, 59, 67, 69, 73,
       74, 79, 81, 82, 85, 86, 89, 96, 97, 98])

In [21]:
filtered_df['Topic Num'].unique()

array([  2,   7,  19,  26,  35,  36,  38,  39,  40, 106, 108,  47, 110,
       113,  52,  55,  56,  59, 130,  67, 136,  69,  73,  74, 142,  79,
       149,  81,  82,  85,  86, 152, 154, 156, 157,  89, 162,  96,  97,
        98, 170, 103, 111, 122, 124, 129, 137, 138, 140, 146, 163, 165,
       169])

In [26]:
final_df2 = pd.concat([filtered_df3,filtered_df4], axis=0)
len(final_df2)

5071

In [28]:
# 엑셀 파일 경로
file_path = "/home/dieselim/main/data/EvMineFinlaResult/240802_이벤트 리스트 수정.xlsx"

# 엑셀 파일을 기존 시트와 함께 열기
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    final_df2.to_excel(writer, sheet_name='뉴스 리스트', index=False)

In [59]:
filtered_df4.loc[filtered_df4['topic number']==145, 'title'].head(30)

0              반도체 고객사 'Made in China' 안 받는다
1            반도체 공장 처음 찾은 바이든"SK실트론은 일류"(종합)
2                 반도체장비 대중 수출 제한, 네덜란드 동참 추진
3              보조금, 신청해도 안해도 고민전문가  대통령도 나서야
4          " 대중 반도체 수출통제 조치, 업체와 기술격차 벌릴 기회"
5            " 인플레 감축법은 인플레를 못 낮춘다" 전문가들의 경고
6                  ", 반도체장비  수출 규제 내달 확대 방침"
7                ", 중국으로 반도체 장비 수출 규제 확대 방침"
8           "10兆 더 든다"…美 추가 건설비용에 우는 삼성·TSMC
9        "TSMC 美공장 투자에 내부 불만 확대…고비용·문화충돌 우려"
10                 "中 봉쇄 문턱 낮춰라" 美 반도체 기업 반기
11                "中 봉쇄 제 발등 찍기" 美 반도체 기업 반기
12               "中핵무기연구소, 수출 통제 피해 美반도체 조달"
13                "中핵무기연구소, 수출통제 피해 美반도체 조달"
14     "尹·바이든 정상회담이 관건"…삼성·SK, 美보조금 신청미루는 이유
15          "美 반도체법 보조금 신청요건 '4대 독소조항' 완화돼야"
16           "美, 삼성·SK 對중국 반도체장비 수출통제 유예 연장"
17          "美국방부, 삼성 기밀 들여다본다" 반도체법 4대 독소조항
18          "美서 반도체 보조금 받아도 中 증산 요구…불확실성 완화"
19         "韓, 공급망 핵심거점으로"  尹·바이든 '반도체 평택선언'
20       "공짜는 없다" 까다로운 美 반도체 보조금에 삼성·하이닉스 고심
21                 "과 동참 협의" 반도체 규제 동맹 압박하는 
22        

# 진짜 최최종

In [9]:
# Excel 파일의 시트를 읽습니다.
df1 = pd.read_excel('/home/dieselim/main/data/EvMineFinlaResult/240808_최종이벤트리스트.xlsx', sheet_name='이벤트 리스트')
df2 = pd.read_excel('/home/dieselim/main/data/EvMineFinlaResult/240808_최종이벤트리스트.xlsx', sheet_name='뉴스 리스트')

In [12]:
filtered_df1 = df1[df1['delete']==0]
filtered_df1.drop('delete', axis=1, inplace=True)
filtered_df1.reset_index(inplace=True, drop=True)
filtered_df1.reset_index(inplace=True, names='Final Event Number')
filtered_df1['Final Event Number']+=1
filtered_df1

/tmp/ipykernel_734566/2002660820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1.drop('delete', axis=1, inplace=True)
/tmp/ipykernel_734566/2002660820.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1['Final Event Number']+=1


,Final Event Number,Event Number,Event Title,Start,End,News Count
0,1,1,삼성-애플 특허전쟁,2011-02-09,2011-10-24,63
1,2,2,마이크론의 엘피다 인수,2012-01-05,2012-08-08,44
2,3,6,도시바 반도체 매각 협상 및 인수전.,2017-02-07,2017-10-24,200
3,4,10,일본의 반도체 소재 수출 규제.,2019-02-17,2020-02-14,494
4,5,11,미국 애리조나주에 TSMC 반도체 공장 설립 및 생산 시작.,2020-05-10,2022-12-09,199
5,6,12,반도체 공급 대란 및 가격 인상.,2020-09-08,2022-12-22,312
6,7,14,미국 정부의 반도체 자료 요구 및 생산수율 공개 압박,2021-07-11,2021-11-29,135
7,8,15,"삼성전자, 미국 텍사스주 테일러시에 20조원 투자하여 반도체 파운드리 제2공장 설립.",2021-09-02,2022-03-10,86
8,9,16,러시아의 우크라이나 침공과 이에 따른 글로벌 반도체 공급망 및 원자재 수급 위기.,2021-11-24,2023-01-17,129
9,10,17,미국 반도체 지원 법안 통과 및 보조금 신청 경쟁,2021-12-08,2023-06-28,160


In [16]:
filtered_df2 = pd.DataFrame()
for i in range(len(filtered_df1)):
    df = df2[df2['event number'] == filtered_df1.loc[i,'Event Number']]
    df['final event number'] = i+1
    filtered_df2 = pd.concat([filtered_df2, df])
filtered_df2

/tmp/ipykernel_734566/2228566049.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final event number'] = i+1
/tmp/ipykernel_734566/2228566049.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final event number'] = i+1
/tmp/ipykernel_734566/2228566049.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,event number,news number,yearmonth,title,sentence,final event number
0,1,1,2011-02-09,[사설] 삼성전자-IBM 특허 공동활용이 의미하는 것,"['대표적인 글로벌 정보기술(IT)기업인 삼성전자와 IBM이 특허 라이선스 계약을 체결했다고 발표했다.', '지금까지 반도체, 통신, 디스플레이, 소프트웨어 등 여러 분야에 걸쳐 특허 협력관계를 유지해 왔던 두 회사가 특허 공동활용으로 협력을 한층 확대 발전시키기로 한 것이다.', '특히 IBM과 삼성전자는 현재 미국 특허등록에서 1위와 2위를 차지하고 있는 업체들이다.', '이번 계약이 정보기술 산업의 혁신을 촉진하는 효과도 가져올 것으로 기대되는 이유다.', '삼성과 IBM의 협력은 경쟁관계인 삼성과 소니, 보완 관계인 삼성과 MS(마이크로소프트) 등과의 특허협력과는 그 성격이 다르다.', '삼성이 이번 계약을 통해 소비자들의 요구와 급변하는 환경에 부합하는 제품 혁신에 나설 것이라고 밝힌 것에서 보듯이 이번 협력은 융합화 등 정보기술의 새로운 트렌드 등장에 선제적으로 대응하자는 데 그 목적이 있음을 시사해 준다.', '또한 삼성전자로서는 IBM과의 포괄적 특허협력 구축으로 특허괴물 등의 예상치 못한 공격으로부터 방어벽을 높이는 효과도 기대했을 수 있다.', '결과적으로 삼성전자는 특허 포트폴리오를 보강함으로써 미래에 대한 불확실성을 제거하고 시장변화에 보다 능동적으로 대응할 수 있게 됐다.', 'IBM 또한 보유특허에 대한 수익 등 기대이익을 충분히 고려했을 게 분명하다.', '이번 협력은 특히 지금 어떤 글로벌기업도 특허문제에서 자유롭지 못한 상황에 처해있음을 보여주고 있다.', 'IT분야의 기술발전 속도가 너무 빨라 독자적인 보유 특허만으로는 제품혁신을 이뤄내기가 그만큼 어려워지고 있기 때문이다.', '특허를 많이 보유하고 있는 기업들 간에도 공유나 라이선스 계약 동기가 생길 수밖에 없는 이유가 바로 여기에 있고, 그런 점에서 미국내 특허등록 1,2위 기업간 라이선스 계약이 갖는 상징적인 의미가 작지 않다.', '국내 다른 IT기업들도 글로벌 시장을 선도하기 위해서는 내부적으로 특허창출 전략을 강화하는 동시에 대외적으로는 특허공유나 라이선스 전략을 적극적으로 구사하는 게 바람직하다.', '한마디로 삼성과 IBM의 라이선스 협력은 특허전략의 의미와 중요성을 새삼 일깨워주고 있다.', '<성공을 부르는 습관> 한국경제신문, 무단 전재 및 재배포 금지<한국온라인신문협회의 디지털뉴스이용규칙에 따른 저작권을 행사합니다', '>']",1
1,1,2,2011-02-09,"삼성전자-IBM, 특허 라이선스 계약 체결","['서울=뉴시스 김정남 기자 = 삼성전자는 미국 IBM과 양사가 보유하고 있는 특허를 상호 사용할 수 있는 특허 크로스 라이선스 계약을 체결했다고 9일 밝혔다.', '양사는 지난 수 십년에 걸쳐 반도체, 통신, 디스플레이, 소프트웨어 등에서 강력한 특허 포트폴리오를 구축해 왔다.', 'IBM과 삼성전자는 지난 수년간 미국 특허 등록수 1, 2위를 차지하고 있다.', '켄 킹 IBM 라이선싱 담당 임원은 ""특허와 혁신은 IBM의 비즈니스 핵심가치""라며 ""이번 계약은 IBM의 글로벌 경쟁력을 높이고, 연구개발(R&D) 투자에 더욱 박차를 가하는 계기가 될 것""이라고 말했다.', '안승호 삼성전자 IP센터 부사장은 ""이번 계약을 통해 다양한 기술을 확보해 고객들의 요구를 반영한 제품과 서비스를 보다 효율적으로 제공할 수 있게 됐다""고 말했다.', 'surrender@newsis.com[뉴시스 이시간 핫 뉴스]ㆍ 격정소나타 최고은 감독, 생활고로 사망 32세ㆍ 하리수 ""미키정, 임신할수있는 여자 만나라""ㆍ 숫자로 보는 2011년 대한민국ㆍ 뉴라이프론, 서민대출시장의 대안 될까ㆍ 양띠, 일 안풀리니 집안도 시끄러워지고 손 안에서 보는 세상, 모바일 뉴시스<저작권자 \'한국언론 뉴스허브\' 뉴시스통신사. 무단전재-재배포 금지.', '>']",1
2,1,3,2011-02-09,"삼성전자·IBM, 특허 라이선스 계약 체결","['[서울신문NTN 김수연 기자] 삼성전자와 IBM이 보유 특허를 상호 사용할 수 있게 됐다.', ""삼성전자는 미국 IBM과 양사가 보유하고 있는 특허를 상호 사용할 수 있는 '특허 크로스 라이선스 계약'을 체결했다고 9일(한국시각) 밝혔다."", '양사는 지난 수십년에 걸쳐 반도체, 통신, 디스플레이, 소프트웨어 등 광범위한 기술에 대해 강력한 특허 포트폴리오를 구축해 왔다고 삼성전자 측은 강조했다.', '켄 킹(Ken King) IBM 라이선싱 담당 임원은 ""특허와 혁신은 IBM의 비즈니스 핵심 가치로 삼성전자와의 특허 라이선스 계약은 IBM의 글로벌 경쟁력을 높이고 R&D 투자에 더욱 박차를 가하는 계기가 될 것""이라고 말했다.', '안승호 삼성전자 IP센터 부사장은 ""이번 IBM과의 특허 라이선스 계약을 통해 삼성전자는 다양한 기술을 확보해 고객들의 요구를 반영한 제품과 서비스를 보다 효율적으로 제공할 수 있게 됐다.""고 전했다.', '김수연 기자 newsyouth@seoulntn.com KT, 오지 해양 통신가능 \'위성휴대전화서비스\' 출시 삼성, 광학 10배 줌 콤팩트카메라 PL210 출시 이상수 사장 ""\'아이덴티티 크론\' 국내 50만대 판매 목표"" 다음, \'공식 맛집블로그\' 활용해 \'맛집 책\' 출간 인텔, 결함 \'칩셋\' 다시 공급 ""제조사가 원해""']",1
3,1,4,2011-02-15,애플 제품 팔릴수록 삼성-LG 웃는다?,"['[동아일보]서울 용산구 LG디스플레이 본사 3층에는 아이패드 존(i-pad zone) 이란 특별한 곳이 있다.', '직원들이 자유롭게 담소를 나누거나 커피를 즐기는 카페테리아 한구석에 누구든 이용할 수 있는 아이패드 여러 대가 설치돼 있는 것. 그리고 아이패드 옆에는 이런 문구가 적힌 푯말이 하나 붙어 있다.', '아이패드는 자사( )의 우수한 기술인 IPS(LCD) 로 만들어진 9.7인치(1024 768) 패널이 내장된 태블릿PC입니다.', '이 회사는 신비주의 를 추구하는 애플의 요청에 따라 자신들이 아이패드 액정표시장치(LCD) 패널을 가장 많이 공급하고 있다는 사실을 대외적으로 홍보하지 못한다.', '하지만 적어도 내부적으로는 아이패드 전용 공간을 따로 만들 정도로 애플 납품에 상당한 의미를 부여하고 있는 것이다.', '업계에선 LG디스플레이가 올해 애플로부터 42억 달러(약 4조7100억 원)의 매출을 올릴 것으로 전망하고 있다.', '하지만 LG디스플레이가 앞으로도 애플발 특수( ) 를 지금처럼 계속 누리기는 만만치 않아 보인다.', '애플이 프리미엄 모바일 기기 시장의 핵심으로 떠오르면서 애플을 잡기 위한 전 세계 디스플레이업계의 경쟁이 갈수록 치열해지고 있기 때문이다.', '애플은 지난해 세계적으로 아이폰 4700만 대와 아이패드 1480만 대를 팔았는데 올해는 이보다 각각 60%와 150%가량 급증한 7500만 대의 아이폰과 3650만 대의 아이패드를 판매할 수 있을 것으로 예상된다.', '애플 제품들은 모델 수가 적은 대신 경쟁사보다 가격대가 높아 부품업계로선 대목을 맞은 셈이다.', '특히 최근 애플이 납품 단가를 떨어뜨리는 동시에 날로 급증하는 제품수요를 따라잡기 위해 부품업체 수를 적극 늘리기로 한 것도 파장을 일으키고 있다.', '애플은 지난해 아이폰4와 아이패드를 만들면서 LG로부터 가장 많은 디스플레이 패널을 구입했지만, 새로 나올 아이폰5와 아이패드2에선 LG의 납품 비중을 줄이는 대신에 삼성 LCD를 새로 공급받기로 한 것으로 알려졌다.', '이에 따라 최근 삼성모바일디스플레이는 애플 납품을 겨냥해 기존의 VA 방식(전기를 가하면 디스플레이 안의 액정이 수직으로 움직이는 기술)에서 벗어나 처음으로 IPS 방식(액정이 수평으로 움직이는 기술) 제품인 슈퍼 PLS LCD를 개발했다.', '당초 IPS LCD의 한계(가격이 비싼 것)를 주장하던 삼성이 방향을 튼 것에 대해 업계에선 IPS LCD를 레티나(망막) 디스플레이 라며 칭찬한 애플의 스티브 잡스 최고경영자(CEO)의 입맛에 맞춘 것

In [ ]:
filtered_df1.drop('Event Number', axis=1, inplace=True)
filtered_df2.drop('event number', axis=1, inplace=True)

In [19]:
with pd.ExcelWriter('/home/dieselim/main/data/EvMineFinlaResult/240808_최종이벤트리스트_수정.xlsx') as writer:
    filtered_df1.to_excel(writer, sheet_name='이벤트 리스트', index=False)
    filtered_df2.to_excel(writer, sheet_name='뉴스 리스트', index=False)